In [ ]:
import numpy as np
import pandas as pd
import sklearn
import itertools

## Reading and Separating Data

In [ ]:
df_ml = pd.read_csv("../data/data_ml.csv")

In [ ]:
separation_date = "2013-12-31"
df_train = df_ml.query("date < @separation_date").copy()
df_test = df_ml.query("@separation_date <= date & date < '2018-12-31'").copy()

In [ ]:
columns_to_drop = [
    "stock_id", "date", # non-feature identifiers
    "R1M_Usd", "R3M_Usd", "R6M_Usd", "R12M_Usd", # numerical labels
    "R1M_Usd_C", "R12M_Usd_C" # categorical labels
]

features = list(df_ml.drop(columns=columns_to_drop).columns)

In [ ]:
X_train = df_train[features]
y_train = df_train["R1M_Usd_C"]

In [ ]:
X_test = df_test[features]
y_test = df_test["R1M_Usd_C"]

In [ ]:
rebalance_dates = list(np.sort(df_test["date"].unique()))

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

In [ ]:
def get_fitting_backtesting_results(specified_model):
    # fitting cross-validation results
    cv = sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True)
    scoring = [
        "accuracy",
        "precision",
        "recall",
        "roc_auc"
    ]
    results_cv = cross_validate(specified_model, X_train, y_train, scoring=scoring, cv=cv)
    accuracy_mean = results_cv['test_accuracy'].mean()
    precision_mean = results_cv['test_precision'].mean()
    recall_mean = results_cv['test_recall'].mean()
    roc_auc_mean = results_cv['test_roc_auc'].mean()

    # final fitting
    model = specified_model
    model.fit(X_train, y_train)

    # generating inferences
    df_screen = df_test[["date", "R1M_Usd"]].copy().reset_index(drop=True)
    df_screen["pred_hard"] = model.predict(X_test)
    df_screen["pred_prob"] = model.predict_proba(X_test)[:,1]

    # hard-prediction
    df_monthly_returns = df_screen.query("pred_hard==1").groupby(["date"])[["R1M_Usd"]].mean().reset_index()
    df_monthly_returns['growth_factor'] = 1 + df_monthly_returns['R1M_Usd']
    df_monthly_returns['equity_curve'] = df_monthly_returns['growth_factor'].cumprod()
    annual_return_hp = ((df_monthly_returns['equity_curve'].iloc[-1]) ** (12 / len(df_monthly_returns))) - 1
    sharpe_ratio_hp = (df_monthly_returns["R1M_Usd"].mean() / df_monthly_returns["R1M_Usd"].std()) * np.sqrt(12)
    
    # top-200 trades
    rebalance_dates = list(np.sort(df_screen["date"].unique()))
    lst_trades = []
    for ix_date in rebalance_dates:
        df = df_screen.query("date == @ix_date").sort_values(by="pred_prob", ascending=False).head(200).copy()
        lst_trades.append(df)
    df_trades = pd.concat(lst_trades)
    
    # top-200 return and sharpe-ratio
    df_monthly_returns = df_trades.groupby("date")[["R1M_Usd"]].mean().reset_index()
    df_monthly_returns['growth_factor'] = 1 + df_monthly_returns['R1M_Usd']
    df_monthly_returns['equity_curve'] = df_monthly_returns['growth_factor'].cumprod()
    annual_return_top = ((df_monthly_returns['equity_curve'].iloc[-1]) ** (12 / len(df_monthly_returns))) - 1
    sharpe_ratio_top = (df_monthly_returns["R1M_Usd"].mean() / df_monthly_returns["R1M_Usd"].std()) * np.sqrt(12)

    # putting results into a DataFrame - Random Forest
    df_results = pd.DataFrame({
        "model_family":["random_forest"],
        "model_name":[str(model)],
        "n_estimators":[model.get_params()["n_estimators"]],
        "criterion":[model.get_params()["criterion"]],
        "max_depth":[model.get_params()["max_depth"]],
        "max_features":[model.get_params()["max_features"]],
        "accuracy":[accuracy_mean],
        "precision":[precision_mean],
        "recall":[recall_mean],
        "roc_auc":[roc_auc_mean],
        "hp_ann_ret":[annual_return_hp],
        "hp_sharpe":[sharpe_ratio_hp],
        "top_200_ann_ret":[annual_return_top],
        "top_200_sharpe":[sharpe_ratio_top],
    })

    return df_results

## Logistic Regression

In [ ]:
#get_fitting_backtesting_results(LogisticRegression(penalty=None))

In [ ]:
# #penalty = ["l2"]
# penalty = [None]
# #C = np.logspace(-5, 1, 7)
# solver = ["lbfgs", "newton-cg", "newton-cholesky", "sag", "saga"]
# results = []
# ix_counter = 1
# for ix_param in itertools.product(penalty, solver):
#     #specified_model = LogisticRegression(penalty=ix_param[0], C=ix_param[1], solver=ix_param[2], max_iter=10000, n_jobs=-1)
#     specified_model = LogisticRegression(penalty=ix_param[0], solver=ix_param[1], max_iter=10000, n_jobs=-1)
#     print(ix_counter, specified_model)
#     df = get_fitting_backtesting_results(specified_model)
#     results.append(df)
#     ix_counter +=  1
# df_results = pd.concat(results)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df_result = get_fitting_backtesting_results(RandomForestClassifier(max_depth=5, n_jobs=-1))

In [ ]:
df_result

,model_family,model_name,n_estimators,criterion,max_depth,max_features,accuracy,precision,recall,roc_auc,hp_ann_ret,hp_sharpe,top_200_ann_ret,top_200_sharpe
0,random_forest,"RandomForestClassifier(max_depth=5, n_jobs=-1)",100,gini,5,sqrt,0.518435,0.514296,0.56048,0.528986,0.092695,0.670272,0.155708,0.818348


In [ ]:
n_estimators = [200] #[25, 50, 100, 150, 200]
criterion = ["gini", "entropy", "log_loss"]
max_depth = [2, 4, 6, 8, 10, 12, 14]
max_features = ["sqrt", "log2", None, 0.1, 0.25, 0.5, 0.75]
results = []
ix_counter = 1
for ix_param in itertools.product(n_estimators, criterion, max_depth, max_features):
    specified_model = RandomForestClassifier(
        n_estimators=ix_param[0],
        criterion=ix_param[1],
        max_depth=ix_param[2],
        max_features=ix_param[3],
        n_jobs=-1
    )
    print(ix_counter, specified_model)
    df = get_fitting_backtesting_results(specified_model)
    results.append(df)
    ix_counter +=  1
df_results = pd.concat(results)

1 RandomForestClassifier(max_depth=2, n_estimators=200, n_jobs=-1)
2 RandomForestClassifier(max_depth=2, max_features='log2', n_estimators=200,
                       n_jobs=-1)
3 RandomForestClassifier(max_depth=2, max_features=None, n_estimators=200,
                       n_jobs=-1)
4 RandomForestClassifier(max_depth=2, max_features=0.1, n_estimators=200,
                       n_jobs=-1)
5 RandomForestClassifier(max_depth=2, max_features=0.25, n_estimators=200,
                       n_jobs=-1)
6 RandomForestClassifier(max_depth=2, max_features=0.5, n_estimators=200,
                       n_jobs=-1)
7 RandomForestClassifier(max_depth=2, max_features=0.75, n_estimators=200,
                       n_jobs=-1)
8 RandomForestClassifier(max_depth=4, n_estimators=200, n_jobs=-1)
9 RandomForestClassifier(max_depth=4, max_features='log2', n_estimators=200,
                       n_jobs=-1)
10 RandomForestClassifier(max_depth=4, max_features=None, n_estimators=200,
                       n_

In [ ]:
# df_results = pd.concat(results)
df_results

,model_family,model_name,n_estimators,criterion,max_depth,max_features,accuracy,precision,recall,roc_auc,hp_ann_ret,hp_sharpe,top_200_ann_ret,top_200_sharpe
0,random_forest,"RandomForestClassifier(max_depth=2, n_estimato...",200,gini,2,sqrt,0.514324,0.509960,0.594869,0.522400,0.087540,0.645197,0.118926,0.718225
0,random_forest,"RandomForestClassifier(max_depth=2, max_featur...",200,gini,2,log2,0.514592,0.510813,0.557676,0.521978,0.088023,0.641906,0.096846,0.596491
0,random_forest,"RandomForestClassifier(max_depth=2, max_featur...",200,gini,2,None,0.513708,0.509954,0.557960,0.522090,0.095047,0.682197,0.104461,0.658980
0,random_forest,"RandomForestClassifier(max_depth=2, max_featur...",200,gini,2,0.1,0.513996,0.509811,0.583948,0.521751,0.084771,0.641491,0.166811,0.893884
0,random_forest,"RandomForestClassifier(max_depth=2, max_featur...",200,gini,2,0.25,0.512693,0.508211,0.611379,0.522015,0.084072,0.634713,0.108581,0.693401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,random_forest,"RandomForestClassifier(criterion='log_loss', m...",200,log_loss,14,None,0.521218,0.517583,0.544123,0.530903,0.106362,0.733212,0.166454,0.878000
0,random_forest,"RandomForestClassifier(criterion='log_loss', m...",200,log_loss,14,0.1,0.519885,0.516207,0.545982,0.527546,0.100663,0.702249,0.167569,0.879367
0,random_forest,"RandomForestClassifier(criterion='log_loss', m...",200,log_loss,14,0.25,0.520946,0.517296,0.544113,0.529342,0.105437,0.732398,0.172205,0.914878
0,random_forest,"RandomForestClassifier(criterion='log_loss', m...",200,log_loss,14,0.5,0.520425,0.516762,0.544407,0.528302,0.103681,0.717881,0.180005,0.973009


In [ ]:
# df_results.plot(x="accuracy", y="top_200_ann_ret", kind="scatter");

In [ ]:
# df_results.plot(x="precision", y="top_200_ann_ret", kind="scatter");

In [ ]:
# df_results.plot(x="recall", y="top_200_ann_ret", kind="scatter");

In [ ]:
# df_results.plot(x="roc_auc", y="top_200_ann_ret", kind="scatter");

In [ ]:
#np.logspace(-5, 2, 8)
#df_results.to_csv("../data/many_models_random_forest_200.csv", index=False)

In [ ]:
# penalty = ["l1", "l2"]
# C = np.logspace(-5, 2, 8)
# solver = ["lbfgs", "newton-cg", "newton-cholesky", "sag"]

# lst = []
# for ix_parameters in itertools.product(penalty, C, solver):
#     lst.append(ix_parameters)
# len(lst)

In [ ]:
# df_results["hp_ann_ret"].hist()

In [ ]:
# df_results["top_200_ann_ret"].hist(bins=50);

In [ ]:
# df_results[["top_200_ann_ret"]].describe()

In [ ]:
#LogisticRegression(C=10.0, max_iter=10000, n_jobs=-1, solver='sag')
#LogisticRegression(C=10.0, max_iter=10000, n_jobs=-1, solver='saga')